<img src="./assets/logo_sizinglab.png" style="float:right; max-width: 60px; display: inline" alt="SizingLab" /></a>

# Motor selection
*Written by Marc Budinger (INSA Toulouse), Scott Delbecq (ISAE-SUPAERO) and Félix Pollet (ISAE-SUPAERO), Toulouse, France.*

## Design graph 

The following diagram represents the design graph of the motor’s selection. The mean speed/thrust (Ωmoy & Tmoy), the max speed/thrust (Ωmax & Tmax) and the battery voltage are assumed to be known here.



```{figure} ./assets/design_graphs/DesignGraphs_motor.svg
:name: design_graph_motor
:align: center
:width: 50%

Motor design graph
```

The design graphs for the overall drone system can be found in [here](./assets/design_graphs/DesignGraphs_Overall_Solution.pdf).


### Sizing code and optimization


In [ ]:
# Specifications

# Reference parameters for scaling laws
# Motor reference
# Ref : AXI 5325/16 GOLD LINE
T_nom_mot_ref = 2.32  # [N.m] rated torque
T_max_mot_ref = 85./70.*T_nom_mot_ref # [N.m] max torque
R_mot_ref = 0.03  # [Ohm] resistance
M_mot_ref = 0.575 # [kg] mass
K_T_ref = 0.03 # [N.m/A] torque coefficient
T_mot_fr_ref = 0.03 # [N.m] friction torque (zero load, nominal speed)

# Assumption
T_pro_to=0.5#[N.m] Propeller Torque during takeoff
Omega_pro_to=400.0#[rad/s] Propeller speed during takeoff
T_pro_hov=1.0#[N.m] Propeller Torque during hover
Omega_pro_hov=0.22#[rad/s] Propeller speed during hover
U_bat_est= 14.0#[V] Battery voltage value (estimation)


In [ ]:
#Design variables

#% MOTOR
# --- 
# Nominal torque selection with hover scenario
T_nom_mot = k_mot * T_pro_hov  # [N*m] Motor nominal torque per propeller

# Torque constant selection with take-off scenario
U_bat = k_vb*1.84*P_pro_to**(0.36)  # [V] battery voltage estimation
K_T = U_bat / (k_speed_mot*Omega_pro_to)  # [N*m/A] or [V/(rad/s)] Kt motor

# Estimation models
M_mot = M_mot_ref * (T_nom_mot/T_nom_mot_ref)**(3./3.5)  # [kg] Motor mass
R_mot = R_mot_ref * (T_nom_mot/T_nom_mot_ref)**(-5./3.5)*(K_T/K_T_ref)**2.  # [ohm] motor resistance
T_mot_fr = T_mot_fr_ref * (T_nom_mot/T_nom_mot_ref)**(3./3.5)  # [N*m] Friction torque
T_max_mot = T_max_mot_ref * (T_nom_mot/T_nom_mot_ref)  # [N*m] Max. torque

# Performance in various operating conditions
# Hover current and voltage
I_mot_hov = (T_pro_hov+T_mot_fr) / K_T  # [A] Current of the motor per propeller
U_mot_hov = R_mot*I_mot_hov + Omega_pro_hov*K_T  # [V] Voltage of the motor per propeller
P_el_mot_hov = U_mot_hov*I_mot_hov  # [W] Hover : electrical power
# Takeoff current and voltage
I_mot_to = (T_pro_to+T_mot_fr) / K_T  # [A] Current of the motor per propeller
U_mot_to = R_mot*I_mot_to + Omega_pro_to*K_T  # [V] Voltage of the motor per propeller
P_el_mot_to = U_mot_to*I_mot_to  # [W] Takeoff : electrical power